In [ ]:
# Path ORAM 
from collections import defaultdict, deque
import random

class Node(object): 
	def __init__(self, ary, idx, left, right): 
		self.val = ary
		self.idx = idx
		self.left = left 
		self.right = right
		
	def __repr__(self):
		return f"({self.idx},{self.val},{self.left},{self.right})"


# constructing a simple binary tree
# actually each of the int in the arrary should be a pointer to a block of size B
#    0    l0
#   /\
#  1  2   l1
#  /\ /\
# 3 4 5 6 l2

n3 = Node([("a", 1),("b", 2),("c", 3),("d", 4)], 3, None, None) 
n4 = Node([("e", 5),("f", 6),("g", 7),("h", 8)], 4, None, None)
n5 = Node([("i", 9),("j", 10),("k", 11),("l", 12)], 5, None, None)
n6 = Node([("m", 13),("n", 14),("o", 15),("p", 16)], 6, None, None)

n1 = Node([("", 0),("", 0),("", 0),("", 0)], 1, n3, n4)
n2 = Node([("", 0),("", 0),("", 0),("", 0)], 2, n5, n6)

n0 = Node([("", 0),("", 0),("", 0),("", 0)], 0, n1, n2)

print(n0)

(0,[('', 0), ('', 0), ('', 0), ('', 0)],(1,[('', 0), ('', 0), ('', 0), ('', 0)],(3,[('a', 1), ('b', 2), ('c', 3), ('d', 4)],None,None),(4,[('e', 5), ('f', 6), ('g', 7), ('h', 8)],None,None)),(2,[('', 0), ('', 0), ('', 0), ('', 0)],(5,[('i', 9), ('j', 10), ('k', 11), ('l', 12)],None,None),(6,[('m', 13), ('n', 14), ('o', 15), ('p', 16)],None,None)))


In [ ]:
for i in range(4):
  temp = []
  for j in range(4):
    temp.append(4 * i + 1 + j)
  print(temp)

[1, 2, 3, 4]
[5, 6, 7, 8]
[9, 10, 11, 12]
[13, 14, 15, 16]


In [ ]:
def getHeight(root):
	if not root: 
		return 0 

	leftHeight = getHeight(root.left)
	rightHeight = getHeight(root.right) 

	return max(leftHeight, rightHeight) + 1 

In [ ]:
LEVELS = getHeight(n0)
print(LEVELS)

3


In [ ]:
N = 28 
Z = 4 # Z numbers of blocks within each bucket
STASH = []
position = defaultdict(int, {k:random.randrange((pow(2, LEVELS - 1) - 1), (pow(2, LEVELS) - 1)) for k in range(N)})

In [ ]:
print(position)

defaultdict(<class 'int'>, {0: 3, 1: 4, 2: 6, 3: 5, 4: 3, 5: 4, 6: 6, 7: 4, 8: 6, 9: 5, 10: 3, 11: 5, 12: 5, 13: 5, 14: 6, 15: 5, 16: 3, 17: 6, 18: 3, 19: 6, 20: 5, 21: 3, 22: 4, 23: 3, 24: 5, 25: 5, 26: 4, 27: 3})


In [ ]:
# should I assume that this is a perfect btree? No 
def getPath(root, NodeId): 
	if not root:
		return []
	
	if root.idx == NodeId:
		return [(root.idx, root.val)] 

	leftPath = getPath(root.left, NodeId) 
	rightPath = getPath(root.right, NodeId)
 
	if leftPath:
		return leftPath +  [(root.idx, root.val)]
	if rightPath: 
		return rightPath + [(root.idx, root.val)]
	return []

In [ ]:
print(getPath(n0, 6))

[(6, [('m', 13), ('n', 14), ('o', 15), ('p', 16)]), (2, [('', 0), ('', 0), ('', 0), ('', 0)]), (0, [('', 0), ('', 0), ('', 0), ('', 0)])]


In [ ]:
# P(x, l) = node
def getNode(root, index, currlevel, tgtlevel): 
	if root is None:
		return None 

	if root.idx == index and currlevel == tgtlevel: 
		return root 

	nextlevel = currlevel + 1

	leftNode = getNode(root.left, index, nextlevel, tgtlevel)
	if leftNode is not None:
		return leftNode

	rightNode = getNode(root.right, index, nextlevel, tgtlevel)
	if rightNode is not None:
		return rightNode

	return None

In [ ]:
print(n3.val[1][1])

2


In [ ]:
def getBlock(root, blockId):
  if root is not None:
    print(root.val)
    for i in range(Z):
      if(root.val):
        if(root.val[i][1] == blockId):
          return root.val[i][0]

    lres = getBlock(root.left, blockId)
    if lres is not None: 
      return lres 

    rres = getBlock(root.right, blockId)
    if rres is not None:
      return rres

getBlock(n0,11)

[('', 0), ('', 0), ('', 0), ('', 0)]
[('', 0), ('', 0), ('', 0), ('', 0)]
[('a', 1), ('b', 2), ('c', 3), ('d', 4)]
[('e', 5), ('f', 6), ('g', 7), ('h', 8)]
[('', 0), ('', 0), ('', 0), ('', 0)]
[('i', 9), ('j', 10), ('k', 11), ('l', 12)]


'k'

In [ ]:
def getNodeAtLevel(root, leafidx, level):
  path = getPath(root, leafidx) #[3,1,0] 
  return path[LEVELS-level-1]


In [ ]:
def writeBackNodes(root, index, currlevel, tgtlevel, data):
	if root is None:
		return None

	if root.idx == index and currlevel == tgtlevel:
		retVal = root.val 
		root.val = data 
		return retVal

	nextlevel = currlevel + 1

	leftTree = writeBackNodes(root.left, index, nextlevel, tgtlevel, data)
	if leftTree: 
		return leftTree
  
	rightTree = writeBackNodes(root.right, index, nextlevel, tgtlevel, data)
	if rightTree:
		return rightTree

	return None  

In [ ]:
def getCandidateBlocks(root, leaf, blockID, level): 
							# root, index(leaf), curl lvl, tgt lvl
	lhs = getNodeAtLevel(root, leaf, level)
	rhs = getNodeAtLevel(root, position[blockID], level)
 
	if lhs is not None and rhs is not None:
		if lhs == rhs:
			return lhs
		else:
			return None 
	return None

In [ ]:
print(position[23])  
tp0 = getCandidateBlocks(n0, 4, 23, 1)
print(tp0)

3
(1, [('', 0), ('', 0), ('', 0), ('', 0)])


In [ ]:
def access(root, opCode, blockId, dataNew): 
	leafIdx = position[blockId]
	print("23 maps to leaf" + str(leafIdx))
	position[blockId] = random.randrange((pow(2, LEVELS - 1) - 1), (pow(2, LEVELS) - 1))
	
	STASH = getPath(root, leafIdx)
	print("Current STASH is ")
	print(STASH)
 
	# read block a from STASH
	dataOld = getBlock(root, blockId)
	print("old data is ")
	print(dataOld)
 
	if opCode == "wr": # assume blockId is in range or define the number of buckets
		# (check if the blockId is valid): do we assume a perfect binary tree? 
		# STASH = [item for item in STASH if item not in [STASH[blockId]]]
		updateNode = getNode(root, leafIdx, 0, LEVELS - 1)
		STASH[0] = dataNew
 
	# write path 
	for l in reversed(range(LEVELS)):
		# insert additional blocks from the STASH to the path along the tree 
		candidateBlocks = []
		for idb in range(N):
				candidateBlocks.append(getCandidateBlocks(root, leafIdx, idb, l))
		# print("Candi Block at level l is")
		# print(candidateBlocks)
	
		if (len(candidateBlocks) >= Z):
			writeBackSize = Z
		else:
			writeBackSize = len(candidateBlocks)

		writeBackBlocks = candidateBlocks[:writeBackSize]
		updatedStash = candidateBlocks[writeBackSize:]

		writeBackNodes(root, leafIdx, 0, l, writeBackBlocks)
	return dataOld
	

In [ ]:
oldD = access(n0, "wr", 23, "x")
print(oldD)

Current STASH is 
[(6, [('m', 13), ('n', 14), ('o', 15), ('p', 16)]), (2, [('', 0), ('', 0), ('', 0), ('', 0)]), (0, [('', 0), ('', 0), ('', 0), ('', 0)])]
[('', 0), ('', 0), ('', 0), ('', 0)]
[('', 0), ('', 0), ('', 0), ('', 0)]
[('a', 1), ('b', 2), ('c', 3), ('d', 4)]
[('e', 5), ('f', 6), ('g', 7), ('h', 8)]
[('', 0), ('', 0), ('', 0), ('', 0)]
[('i', 9), ('j', 10), ('k', 11), ('l', 12)]
[('m', 13), ('n', 14), ('o', 15), ('p', 16)]
old data is 
None
None
